In [ ]:
# Imports
import os
import PyPDF2
import re
import pandas as pd

In [ ]:
# Regulärer Ausdruck für die Erkennung der Ausgaben
def extract_journal_issue(text):
    pattern = r'(Nº|N°)?\s*(\d{1,2}\/\d{2,4})'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(2).strip() # Heftnummer extrahieren
    return None

In [ ]:
# Funktion zum extrahieren von Informationen aus PDF Dateien
def extract_lines_with_keywords(pdf_path, heading, keyword_mapping):
    results = {}
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file) # PyPDF2 Reader zum Einlesen der PDF Dateien
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num] # aktuelle Seite
            text = page.extract_text()
            # Suche Titelseite
            if re.search(heading, text, re.IGNORECASE):
                journal_issue = extract_journal_issue(text)
                # Iteration über die Aktiennamen und deren Synonyme aus dem Mapping
                for stock_name, variants in keyword_mapping.items():
                    # Regulärer Ausdruck, um Synonyme zu suchen und naheliegende Seitenzahlen zu finden
                    pattern = re.compile(r'(\b(?:' + '|'.join([re.escape(variant) for variant in variants]) + r'))(?: \w+)? (\d+\s*,?\s*\d*|\d*\s*,?\s*\d+)\b', re.IGNORECASE)
                    matches = pattern.findall(text)
                    if matches:
                        for match in matches:
                            stock_name, page_number = match
                            if stock_name not in results:
                                results[stock_name] = []
                            results[stock_name].append({'Ausgabe': journal_issue, 'Seite': page_number, 'Text': text})
    return results

In [ ]:
# Funktion zum Iterieren über alle Artikel
def scrape_articles(directory_path, heading, keyword_mapping):
    all_results = {}  
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory_path, filename)
# Ausgeben welche Datei aktuell verarbeitet wird
            print(f"Extrahiere Daten aus {pdf_path}...")
# Festlegen der zu extrahierenden relevanten Daten
            data = extract_lines_with_keywords(pdf_path, heading, keyword_mapping)
            for stock_name, values in data.items():
                # Überprüfen, ob der Aktienname bereits im all_results-Dict existiert
                if stock_name not in all_results:
                    # Wenn nicht, füge den Aktienname hinzu
                    all_results[stock_name] = []
                # Füge extrahierte Daten für die Aktie zur Liste hinzu
                all_results[stock_name].extend(values)
    return all_results

In [11]:
directory_path = 'articles'
heading = 'Alle ausgewählten Aktien in diesem Heft'

stock_mapping = {
    'Shop Apotheke': ['Shop Apotheke', 'Redcare Pharmacy'],
    'Aixtron': ['AIXTRON', 'aixtron'],
    'Energiekontor': ['Energiekontor', 'Energiekontor AG'],
    'Hugo Boss': ['HUGO BOSS'],
    'Sartorius AG': ['Sartorius', 'Sartorius Vz.', 'Sartorius AG']
}

In [24]:
# Durchsuche alle Artikel im Verzeichnis und speichere die Daten als Dictionary
data_dict = scrape_articles(directory_path, heading, stock_mapping)

# Erstelle ein DataFrame aus dem Dictionary
rows = []
for stock_name, data in data_dict.items():
    for entry in data:
        row = {
            'Aktie': stock_name,
            'Ausgabe': entry['Ausgabe'],
            'Seite': entry['Seite'],
            'Text': entry['Text']
        }
        rows.append(row)
dataframe = pd.DataFrame(rows)

# Speichern des DataFrames als Excel-Datei für manuelle Extraktion
dataframe.to_excel('./sentiment_data/stock_data.xlsx', index=False)

Extrahiere Daten aus articles\AKTIONR0120Sonderheft.pdf...
Extrahiere Daten aus articles\AKTIONR0220.pdf...
Extrahiere Daten aus articles\AKTIONR52190120.pdf...
Extrahiere Daten aus articles\DA_1952-2001.pdf...
Extrahiere Daten aus articles\DA_2002.pdf...
Extrahiere Daten aus articles\DA_2003.pdf...
Extrahiere Daten aus articles\DA_2004.pdf...
Extrahiere Daten aus articles\DA_2005.pdf...
Extrahiere Daten aus articles\DA_2006.pdf...
Extrahiere Daten aus articles\DA_2007.pdf...
Extrahiere Daten aus articles\DA_2008.pdf...
Extrahiere Daten aus articles\DA_2009.pdf...
Extrahiere Daten aus articles\DA_2010.pdf...
Extrahiere Daten aus articles\DA_2011.pdf...
Extrahiere Daten aus articles\DA_2012.pdf...
Extrahiere Daten aus articles\DA_2013.pdf...
Extrahiere Daten aus articles\DA_2014.pdf...
Extrahiere Daten aus articles\DA_2015.pdf...
Extrahiere Daten aus articles\DA_2016.pdf...
Extrahiere Daten aus articles\DA_2017.pdf...
Extrahiere Daten aus articles\DA_2018.pdf...
Extrahiere Daten aus art